# Carboxylic Acids – RDKit K-based Clustering POC

This notebook loads carboxylic acids from `All Carboxylic Acids.xlsx`, computes RDKit descriptors, performs K-means clustering, and provides interactive K control and molecule inspection to support a future dashboard/web app.

Sections:
- Setup and data loading
- RDKit descriptor generation
- Scaling, dimensionality reduction, and K-means clustering utilities
- K vs. clustering quality ("accuracy") plots
- Interactive cluster explorer (K slider, point/cluster selection)
- Structure rendering for selected molecules


## 1. Setup and imports

Run the next cell once per session to import the required libraries and configure the path to the Excel file.

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

!pip install rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors, Draw
from rdkit.Chem.Draw import IPythonConsole  # enables rich molecule reprs in notebooks

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors

import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import IntSlider, Output, VBox, HBox, Layout

# Path to the Excel file in this project directory
DATA_PATH = Path("All Carboxylic Acids.xlsx")
assert DATA_PATH.exists(), f"Expected Excel file not found at {DATA_PATH!s}"

DATA_PATH

PosixPath('All Carboxylic Acids.xlsx')

## 2. Load and clean data

This cell reads the Excel file, automatically finds the header row that contains `Canonical SMILES`, and keeps only rows with valid SMILES strings.

In [2]:
# Read raw sheet without headers so we can locate the real header row
raw = pd.read_excel(DATA_PATH, header=None)
print("Raw shape:", raw.shape)

# Find the first row that contains the string "Canonical SMILES" in any column
mask_header = raw.apply(lambda col: col.astype(str).str.contains("Canonical SMILES", na=False))
header_row_candidates = np.where(mask_header.any(axis=1))[0]
if len(header_row_candidates) == 0:
    raise RuntimeError("Could not find a header row containing 'Canonical SMILES'")

header_row_idx = int(header_row_candidates[0])
print("Using header row index:", header_row_idx)

# Re-read with the discovered header row
base_df = pd.read_excel(DATA_PATH, header=header_row_idx)
print("Columns:", list(base_df.columns))

# Normalise some convenient column names
COLUMN_NAME = "CA Index Name" if "CA Index Name" in base_df.columns else base_df.columns[1]
COLUMN_SMILES = "Canonical SMILES" if "Canonical SMILES" in base_df.columns else base_df.columns[3]

base_df = base_df.rename(columns={COLUMN_NAME: "Name", COLUMN_SMILES: "SMILES"})

# Keep only rows with a non-null SMILES value
base_df = base_df[base_df["SMILES"].notna()].copy()
base_df["SMILES"] = base_df["SMILES"].astype(str)

# Drop duplicate SMILES if any
base_df = base_df.drop_duplicates(subset=["SMILES"]).reset_index(drop=True)

print(f"Number of molecules with SMILES: {len(base_df)}")
base_df.head()

Raw shape: (6004, 5)
Using header row index: 4
Columns: ['CAS Registry Number', 'CA Index Name', 'Molecular Formula', 'Canonical SMILES', 'Isomeric SMILES']
Number of molecules with SMILES: 5690


,CAS Registry Number,Name,Molecular Formula,SMILES,Isomeric SMILES
0,828-51-3,"1-Adamantanecarboxylic acid (6CI, 7CI, 8CI)",C11H16O2,O=C(O)C12CC3CC(CC(C3)C1)C2,NaN
1,471-53-4,Glycyrrhetinic acid,C30H46O4,O=C(O)C1(C)CCC2(C)CCC3(C(=CC(=O)C4C5(C)CCC(O)C...,C[C@]12[C@@]([C@]3(C)[C@@](CC1)(C(C)(C)[C@@H](...
2,25812-30-0,Gemfibrozil,C15H22O3,O=C(O)C(C)(C)CCCOC1=CC(=CC=C1C)C,NaN
3,508-02-1,Oleanolic acid,C30H48O3,O=C(O)C12CCC(C)(C)CC2C3=CCC4C5(C)CCC(O)C(C)(C)...,C[C@]12[C@@]3(C)C([C@]4([C@@](C(O)=O)(CC3)CCC(...
4,42711-75-1,3-Hydroxy-1-adamantanecarboxylic acid,C11H16O3,O=C(O)C12CC3CC(CC(O)(C3)C1)C2,NaN


## 3. RDKit molecules and descriptor calculation

This section converts SMILES to RDKit molecules, filters out invalid structures, and computes a descriptor matrix for clustering.

In [3]:
# Prepare RDKit molecules
def smiles_to_mol(smiles: str):
    """Convert SMILES to an RDKit Mol, returning None on failure."""
    if not isinstance(smiles, str) or not smiles.strip():
        return None
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        Chem.SanitizeMol(mol)
        return mol
    except Exception:
        return None

mols = base_df["SMILES"].apply(smiles_to_mol)
valid_mask = mols.notnull()
print(f"Valid RDKit molecules: {valid_mask.sum()} / {len(base_df)}")

mol_df = base_df.loc[valid_mask].copy().reset_index(drop=True)
mol_df["RDKitMol"] = mols[valid_mask].reset_index(drop=True)

# Define descriptor functions from RDKit
_descriptor_list = Descriptors._descList
DESCRIPTOR_NAMES = [d[0] for d in _descriptor_list]
DESCRIPTOR_FUNCS = [d[1] for d in _descriptor_list]
print(f"Using {len(DESCRIPTOR_NAMES)} RDKit descriptors")


def compute_descriptors_for_mol(mol):
    """Return a list of descriptor values for a single Mol, with NaN on failure."""
    values = []
    for fn in DESCRIPTOR_FUNCS:
        try:
            values.append(fn(mol))
        except Exception:
            values.append(np.nan)
    return values


# Compute descriptor matrix
rows = []
for i, mol in enumerate(mol_df["RDKitMol"]):
    if (i + 1) % 500 == 0:
        print(f"Computed descriptors for {i + 1} molecules...")
    rows.append(compute_descriptors_for_mol(mol))

desc_df = pd.DataFrame(rows, columns=DESCRIPTOR_NAMES)

# Combine with molecule metadata
desc_df.index = mol_df.index
mol_desc_df = pd.concat([mol_df, desc_df], axis=1)

print("Final descriptor frame shape:", mol_desc_df.shape)
mol_desc_df.head()

Valid RDKit molecules: 5690 / 5690
Using 217 RDKit descriptors
Computed descriptors for 500 molecules...
Computed descriptors for 1000 molecules...
Computed descriptors for 1500 molecules...
Computed descriptors for 2000 molecules...
Computed descriptors for 2500 molecules...
Computed descriptors for 3000 molecules...
Computed descriptors for 3500 molecules...
Computed descriptors for 4000 molecules...
Computed descriptors for 4500 molecules...
Computed descriptors for 5000 molecules...
Computed descriptors for 5500 molecules...
Final descriptor frame shape: (5690, 223)


,CAS Registry Number,Name,Molecular Formula,SMILES,Isomeric SMILES,RDKitMol,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,828-51-3,"1-Adamantanecarboxylic acid (6CI, 7CI, 8CI)",C11H16O2,O=C(O)C12CC3CC(CC(C3)C1)C2,NaN,<rdkit.Chem.rdchem.Mol object at 0x142505a80>,11.252917,11.252917,0.282986,-0.507778,...,0,0,0,0,0,0,0,0,0,0
1,471-53-4,Glycyrrhetinic acid,C30H46O4,O=C(O)C1(C)CCC2(C)CCC3(C(=CC(=O)C4C5(C)CCC(O)C...,C[C@]12[C@@]([C@]3(C)[C@@](CC1)(C(C)(C)[C@@H](...,<rdkit.Chem.rdchem.Mol object at 0x142505d90>,14.202352,14.202352,0.029556,-0.711072,...,0,0,0,0,0,0,0,0,0,0
2,25812-30-0,Gemfibrozil,C15H22O3,O=C(O)C(C)(C)CCCOC1=CC(=CC=C1C)C,NaN,<rdkit.Chem.rdchem.Mol object at 0x142505f50>,10.943173,10.943173,0.556551,-0.755337,...,0,0,0,0,0,0,0,0,0,0
3,508-02-1,Oleanolic acid,C30H48O3,O=C(O)C12CCC(C)(C)CC2C3=CCC4C5(C)CCC(O)C(C)(C)...,C[C@]12[C@@]3(C)C([C@]4([C@@](C(O)=O)(CC3)CCC(...,<rdkit.Chem.rdchem.Mol object at 0x142b8d2a0>,12.753229,12.753229,0.030113,-0.553469,...,0,0,0,0,0,0,0,0,0,0
4,42711-75-1,3-Hydroxy-1-adamantanecarboxylic acid,C11H16O3,O=C(O)C12CC3CC(CC(O)(C3)C1)C2,NaN,<rdkit.Chem.rdchem.Mol object at 0x142b8d620>,11.284028,11.284028,0.469769,-0.675903,...,0,0,0,0,0,0,0,0,0,0


## 4. Scaling, dimensionality reduction, and K-means clustering utilities

These helpers standardize the descriptor space, compute a 2D embedding for plotting, and run K-means clustering for any chosen K.

In [4]:
# Feature matrix: use all RDKit numeric descriptors
FEATURE_COLUMNS = DESCRIPTOR_NAMES

# Drop columns that are entirely NaN (if any)
all_nan_cols = [c for c in FEATURE_COLUMNS if mol_desc_df[c].isna().all()]
if all_nan_cols:
    print("Dropping all-NaN descriptor columns:", all_nan_cols)
    FEATURE_COLUMNS = [c for c in FEATURE_COLUMNS if c not in all_nan_cols]


def prepare_feature_matrix(df, feature_cols=FEATURE_COLUMNS):
    """Return standardized feature matrix X_scaled and the fitted scaler."""
    X = df[feature_cols].astype(float).values
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, scaler


def fit_pca_embedding(X_scaled, n_components=2, random_state=0):
    """Fit PCA and return 2D coordinates and the PCA model."""
    pca = PCA(n_components=n_components, random_state=random_state)
    coords = pca.fit_transform(X_scaled)
    return coords, pca


def run_kmeans(X_scaled, k, random_state=0):
    """Run K-means for K clusters and return labels and model."""
    model = KMeans(n_clusters=k, random_state=random_state, n_init="auto")
    labels = model.fit_predict(X_scaled)
    return labels, model


# Build the standardized feature matrix and 2D embedding
X_scaled, scaler = prepare_feature_matrix(mol_desc_df, FEATURE_COLUMNS)
embedding_2d, pca_model = fit_pca_embedding(X_scaled, n_components=2, random_state=0)

mol_desc_df["embed_x"] = embedding_2d[:, 0]
mol_desc_df["embed_y"] = embedding_2d[:, 1]

print("X_scaled shape:", X_scaled.shape)
print("First few embedded points:")
mol_desc_df[["Name", "embed_x", "embed_y"]].head()

X_scaled shape: (5690, 217)
First few embedded points:


,Name,embed_x,embed_y
0,"1-Adamantanecarboxylic acid (6CI, 7CI, 8CI)",-1.572552,-7.130195
1,Glycyrrhetinic acid,15.139165,-7.109108
2,Gemfibrozil,-1.386065,2.447415
3,Oleanolic acid,13.684420,-8.039666
4,3-Hydroxy-1-adamantanecarboxylic acid,-0.391916,-7.480190


## 5. K vs. clustering quality ("accuracy") plots

This cell computes inertia and silhouette scores across a range of K values so you can see which K values give tighter, well-separated clusters.

In [5]:
def evaluate_k_range(X_scaled, k_values):
    records = []
    for k in k_values:
        labels, model = run_kmeans(X_scaled, k)
        inertia = model.inertia_
        sil = silhouette_score(X_scaled, labels) if k > 1 else np.nan
        records.append({"k": k, "inertia": inertia, "silhouette": sil})
    return pd.DataFrame(records)


K_VALUES = list(range(2, 16))  # you can tweak this range
metrics_df = evaluate_k_range(X_scaled, K_VALUES)
metrics_df

fig_inertia = px.line(metrics_df, x="k", y="inertia", markers=True, title="Inertia vs K")
fig_silhouette = px.line(metrics_df, x="k", y="silhouette", markers=True, title="Silhouette score vs K")

fig_inertia.show()
fig_silhouette.show()

## 6. Interactive cluster explorer (K slider, point selection, neighbors)

Use this UI to:
- Choose the number of clusters K with a sidebar-style slider.
- Explore an interactive 2D embedding where each point is a molecule.
- Click a point to see its name, descriptors, nearest neighbors, and RDKit structure.

In [6]:
# Prepare nearest-neighbor model in descriptor space
nn_model = NearestNeighbors(n_neighbors=6, metric="euclidean")  # 1 + 5 nearest neighbors
nn_model.fit(X_scaled)

# Widgets for interactive UI
k_slider = IntSlider(
    value=8,
    min=2,
    max=20,
    step=1,
    description="K clusters",
    continuous_update=False,
    style={"description_width": "initial"},
)

plot_out = Output(layout=Layout(width="800px", height="650px"))
detail_out = Output(layout=Layout(width="500px"))


def make_figure_and_labels(k):
    """Compute K-means labels for K, attach to frame, and return a FigureWidget."""
    labels, model = run_kmeans(X_scaled, k)
    mol_desc_df["cluster"] = labels

    fig = go.FigureWidget(
        data=[
            go.Scattergl(
                x=mol_desc_df["embed_x"],
                y=mol_desc_df["embed_y"],
                mode="markers",
                marker=dict(
                    color=mol_desc_df["cluster"],
                    colorscale="Viridis",
                    showscale=True,
                    size=6,
                    opacity=0.8,
                ),
                text=mol_desc_df["Name"],
                customdata=np.arange(len(mol_desc_df)),  # store row index
                hovertemplate="Name: %{text}<br>Cluster: %{marker.color}<extra></extra>",
            )
        ]
    )

    fig.update_layout(
        title=f"Carboxylic acids – 2D embedding with K={k} clusters",
        xaxis_title="PC1",
        yaxis_title="PC2",
        height=600,
        width=800,
    )

    return fig, labels


current_fig, current_labels = make_figure_and_labels(k_slider.value)


def show_details_for_index(idx: int):
    """Populate the right-hand panel with details and structure for one molecule."""
    row = mol_desc_df.iloc[idx]
    with detail_out:
        detail_out.clear_output(wait=True)

        display(pd.DataFrame([
            {
                "Name": row["Name"],
                "Cluster": int(row["cluster"]),
                "SMILES": row["SMILES"],
            }
        ]))

        # Show a compact subset of descriptors (you can expand this if desired)
        key_descs = [
            "MolWt",
            "MolLogP",
            "TPSA",
            "NumHAcceptors",
            "NumHDonors",
            "NumRotatableBonds",
        ]
        key_descs = [d for d in key_descs if d in mol_desc_df.columns]

        if key_descs:
            display(row[key_descs].to_frame().T)

        # Nearest neighbors in descriptor space (excluding itself at position 0)
        distances, indices = nn_model.kneighbors(X_scaled[[idx]])
        neighbor_idx = indices[0][1:]
        neighbors = mol_desc_df.iloc[neighbor_idx][["Name", "cluster", "SMILES"]]
        neighbors = neighbors.reset_index(drop=True)
        display(neighbors.style.set_caption("Nearest neighbors in descriptor space"))

        # RDKit structure for selected molecule
        mol = row["RDKitMol"]
        if mol is not None:
            img = Draw.MolToImage(mol, size=(300, 300))
            display(img)


# Click handler for the scatter plot
scatter = current_fig.data[0]


def handle_click(trace, points, selector):
    if not points.point_inds:
        return
    # We stored the DataFrame index in customdata
    point_index = int(trace.customdata[points.point_inds[0]])
    show_details_for_index(point_index)


scatter.on_click(handle_click)

with plot_out:
    display(current_fig)


def on_k_change(change):
    if change["name"] != "value":
        return
    new_k = int(change["new"])
    new_fig, labels = make_figure_and_labels(new_k)

    # Re-wire click handler on the new figure
    new_scatter = new_fig.data[0]
    new_scatter.on_click(handle_click)

    plot_out.clear_output(wait=True)
    with plot_out:
        display(new_fig)

    global current_fig, current_labels
    current_fig, current_labels = new_fig, labels


k_slider.observe(on_k_change, names="value")

# Layout approximating a sidebar (left) + main plot + right-hand detail pane
ui = HBox([
    VBox([k_slider, plot_out]),
    detail_out,
])

ui

## 7. Reusable helpers for a future dashboard

The functions defined above (`prepare_feature_matrix`, `fit_pca_embedding`, `run_kmeans`, `evaluate_k_range`, and `make_figure_and_labels`) are intentionally written so they can be imported directly into a web app (e.g. Streamlit or Dash) to reproduce this behavior with a sidebar, main plot, and detail pane.